# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [15]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*2, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [16]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [17]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [18]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.8
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Cornell'

In [19]:
all_best_auc = []
all_best_ap = []
for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 164.0261
Epoch: 01, Loss: 163.8805
Epoch: 02, Loss: 163.6052
Epoch: 03, Loss: 163.2160
Epoch: 04, Loss: 162.7276
AP SCORE：  0.6421013180368019
AUC SCORE:  0.6176
Epoch: 00, Loss: 162.1541
Epoch: 01, Loss: 162.1349
Epoch: 02, Loss: 162.0984
Epoch: 03, Loss: 162.0468
Epoch: 04, Loss: 161.9819
AP SCORE：  0.661140425647099
AUC SCORE:  0.6352
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 161.9054
Epoch: 01, Loss: 161.7709
Epoch: 02, Loss: 161.5163
Epoch: 03, Loss: 161.1557
Epoch: 04, Loss: 160.7025
AP SCORE：  0.6709641995179081
AUC SCORE:  0.6496
Epoch: 00, Loss: 160.1691
Epoch: 01, Loss: 160.1519
Epoch: 02, Loss: 160.1195
Epoch: 03, Loss: 160.0734
Epoch: 04, Loss: 160.0154
AP SCORE：  0.6732347593674153
AUC SCORE:  0.6544
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 159.9469
Epoch: 01, Loss: 159.8206
Epoch: 02, Loss: 159.5815
Epoch: 03, Loss: 159.

Epoch: 01, Loss: 134.1207
Epoch: 02, Loss: 133.9870
Epoch: 03, Loss: 133.7971
Epoch: 04, Loss: 133.5574
AP SCORE：  0.770281558824089
AUC SCORE:  0.8128
Epoch: 00, Loss: 133.2740
Epoch: 01, Loss: 133.2570
Epoch: 02, Loss: 133.2246
Epoch: 03, Loss: 133.1785
Epoch: 04, Loss: 133.1203
AP SCORE：  0.7707248212354367
AUC SCORE:  0.8143999999999999
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 133.0514
Epoch: 01, Loss: 132.9829
Epoch: 02, Loss: 132.8529
Epoch: 03, Loss: 132.6683
Epoch: 04, Loss: 132.4353
AP SCORE：  0.7707248212354367
AUC SCORE:  0.8143999999999999
Epoch: 00, Loss: 132.1597
Epoch: 01, Loss: 132.1427
Epoch: 02, Loss: 132.1104
Epoch: 03, Loss: 132.0645
Epoch: 04, Loss: 132.0064
AP SCORE：  0.7752390537770886
AUC SCORE:  0.8192
######################### 模块循环 ： 22 ##########################
Epoch: 00, Loss: 131.9377
Epoch: 01, Loss: 131.8710
Epoch: 02, Loss: 131.7445
Epoch: 03, Loss: 131.5648
Epoch: 04, Loss: 131.3381
AP SCORE：  0.7774728200108547
A

AP SCORE：  0.8406172428139984
AUC SCORE:  0.8752
Epoch: 00, Loss: 115.6503
Epoch: 01, Loss: 115.6365
Epoch: 02, Loss: 115.6101
Epoch: 03, Loss: 115.5728
Epoch: 04, Loss: 115.5257
AP SCORE：  0.8426760663434103
AUC SCORE:  0.8767999999999999
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 115.4700
Epoch: 01, Loss: 115.4261
Epoch: 02, Loss: 115.3428
Epoch: 03, Loss: 115.2245
Epoch: 04, Loss: 115.0749
AP SCORE：  0.8432858224409712
AUC SCORE:  0.8784000000000001
Epoch: 00, Loss: 114.8976
Epoch: 01, Loss: 114.8839
Epoch: 02, Loss: 114.8579
Epoch: 03, Loss: 114.8210
Epoch: 04, Loss: 114.7745
AP SCORE：  0.8432858224409712
AUC SCORE:  0.8784000000000001
######################### 模块循环 ： 41 ##########################
Epoch: 00, Loss: 114.7195
Epoch: 01, Loss: 114.6763
Epoch: 02, Loss: 114.5945
Epoch: 03, Loss: 114.4781
Epoch: 04, Loss: 114.3310
AP SCORE：  0.8446858224409712
AUC SCORE:  0.8800000000000001
Epoch: 00, Loss: 114.1567
Epoch: 01, Loss: 114.1431
Epoch: 02

Epoch: 00, Loss: 103.4637
Epoch: 01, Loss: 103.4337
Epoch: 02, Loss: 103.3767
Epoch: 03, Loss: 103.2956
Epoch: 04, Loss: 103.1930
AP SCORE：  0.9002220972697753
AUC SCORE:  0.9152
Epoch: 00, Loss: 103.0714
Epoch: 01, Loss: 103.0637
Epoch: 02, Loss: 103.0489
Epoch: 03, Loss: 103.0281
Epoch: 04, Loss: 103.0017
AP SCORE：  0.9002220972697753
AUC SCORE:  0.9152
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 102.9706
Epoch: 01, Loss: 102.9410
Epoch: 02, Loss: 102.8849
Epoch: 03, Loss: 102.8050
Epoch: 04, Loss: 102.7039
AP SCORE：  0.9002220972697753
AUC SCORE:  0.9152
Epoch: 00, Loss: 102.5840
Epoch: 01, Loss: 102.5764
Epoch: 02, Loss: 102.5619
Epoch: 03, Loss: 102.5414
Epoch: 04, Loss: 102.5156
AP SCORE：  0.9011898392052592
AUC SCORE:  0.9168
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 102.4850
Epoch: 01, Loss: 102.4558
Epoch: 02, Loss: 102.4004
Epoch: 03, Loss: 102.3216
Epoch: 04, Loss: 102.2218
AP SCORE：  0.902136808902228

AP SCORE：  0.9314185898967464
AUC SCORE:  0.9392
Epoch: 00, Loss: 94.2081
Epoch: 01, Loss: 94.2018
Epoch: 02, Loss: 94.1900
Epoch: 03, Loss: 94.1732
Epoch: 04, Loss: 94.1520
AP SCORE：  0.9314185898967464
AUC SCORE:  0.9392
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 94.1269
Epoch: 01, Loss: 94.1037
Epoch: 02, Loss: 94.0596
Epoch: 03, Loss: 93.9968
Epoch: 04, Loss: 93.9174
AP SCORE：  0.9314185898967464
AUC SCORE:  0.9392
Epoch: 00, Loss: 93.8232
Epoch: 01, Loss: 93.8171
Epoch: 02, Loss: 93.8055
Epoch: 03, Loss: 93.7890
Epoch: 04, Loss: 93.7682
AP SCORE：  0.9314185898967464
AUC SCORE:  0.9392
######################### 模块循环 ： 81 ##########################
Epoch: 00, Loss: 93.7436
Epoch: 01, Loss: 93.7207
Epoch: 02, Loss: 93.6772
Epoch: 03, Loss: 93.6154
Epoch: 04, Loss: 93.5372
AP SCORE：  0.9344955129736696
AUC SCORE:  0.9408
Epoch: 00, Loss: 93.4444
Epoch: 01, Loss: 93.4384
Epoch: 02, Loss: 93.4270
Epoch: 03, Loss: 93.4108
Epoch: 04, Loss: 93.3905
AP S

...

Epoch: 03, Loss: 91.6431
Epoch: 04, Loss: 91.6270
AP SCORE：  0.7601417371645771
AUC SCORE:  0.784
######################### 模块循环 ： 95 ##########################
Epoch: 00, Loss: 91.6080
Epoch: 01, Loss: 91.5911
Epoch: 02, Loss: 91.5589
Epoch: 03, Loss: 91.5132
Epoch: 04, Loss: 91.4554
AP SCORE：  0.7589756512180033
AUC SCORE:  0.7808
Epoch: 00, Loss: 91.3869
Epoch: 01, Loss: 91.3821
Epoch: 02, Loss: 91.3731
Epoch: 03, Loss: 91.3603
Epoch: 04, Loss: 91.3442
AP SCORE：  0.7595610170716618
AUC SCORE:  0.7824
######################### 模块循环 ： 96 ##########################
Epoch: 00, Loss: 91.3252
Epoch: 01, Loss: 91.3084
Epoch: 02, Loss: 91.2765
Epoch: 03, Loss: 91.2311
Epoch: 04, Loss: 91.1737
AP SCORE：  0.756689936932289
AUC SCORE:  0.7792
Epoch: 00, Loss: 91.1057
Epoch: 01, Loss: 91.1009
Epoch: 02, Loss: 91.0919
Epoch: 03, Loss: 91.0791
Epoch: 04, Loss: 91.0631
AP SCORE：  0.7572753027859476
AUC SCORE:  0.7808
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 9

Epoch: 02, Loss: 137.5965
Epoch: 03, Loss: 137.5548
Epoch: 04, Loss: 137.5021
AP SCORE：  0.8024738384237694
AUC SCORE:  0.7711999999999999
######################### 模块循环 ： 16 ##########################
Epoch: 00, Loss: 137.4394
Epoch: 01, Loss: 137.3561
Epoch: 02, Loss: 137.1982
Epoch: 03, Loss: 136.9741
Epoch: 04, Loss: 136.6914
AP SCORE：  0.8089657605367442
AUC SCORE:  0.776
Epoch: 00, Loss: 136.3575
Epoch: 01, Loss: 136.3415
Epoch: 02, Loss: 136.3112
Epoch: 03, Loss: 136.2679
Epoch: 04, Loss: 136.2132
AP SCORE：  0.8118680161758418
AUC SCORE:  0.7791999999999999
######################### 模块循环 ： 17 ##########################
Epoch: 00, Loss: 136.1483
Epoch: 01, Loss: 136.0673
Epoch: 02, Loss: 135.9139
Epoch: 03, Loss: 135.6961
Epoch: 04, Loss: 135.4214
AP SCORE：  0.8118117522909776
AUC SCORE:  0.7807999999999999
Epoch: 00, Loss: 135.0968
Epoch: 01, Loss: 135.0803
Epoch: 02, Loss: 135.0489
Epoch: 03, Loss: 135.0042
Epoch: 04, Loss: 134.9477
AP SCORE：  0.8141002594827107
AUC SCORE:  0.7

Epoch: 01, Loss: 114.7333
Epoch: 02, Loss: 114.6500
Epoch: 03, Loss: 114.5316
Epoch: 04, Loss: 114.3822
AP SCORE：  0.8582835621550793
AUC SCORE:  0.8432
Epoch: 00, Loss: 114.2056
Epoch: 01, Loss: 114.1909
Epoch: 02, Loss: 114.1633
Epoch: 03, Loss: 114.1242
Epoch: 04, Loss: 114.0750
AP SCORE：  0.8582835621550793
AUC SCORE:  0.8432
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 114.0173
Epoch: 01, Loss: 113.9746
Epoch: 02, Loss: 113.8937
Epoch: 03, Loss: 113.7789
Epoch: 04, Loss: 113.6339
AP SCORE：  0.8621742967434882
AUC SCORE:  0.8432
Epoch: 00, Loss: 113.4625
Epoch: 01, Loss: 113.4486
Epoch: 02, Loss: 113.4224
Epoch: 03, Loss: 113.3853
Epoch: 04, Loss: 113.3387
AP SCORE：  0.8627950794695343
AUC SCORE:  0.8448
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 113.2840
Epoch: 01, Loss: 113.2426
Epoch: 02, Loss: 113.1642
Epoch: 03, Loss: 113.0528
Epoch: 04, Loss: 112.9122
AP SCORE：  0.8612931031849491
AUC SCORE:  0.8432
Epoch

AP SCORE：  0.8700090020746369
AUC SCORE:  0.864
######################### 模块循环 ： 59 ##########################
Epoch: 00, Loss: 102.8230
Epoch: 01, Loss: 102.7956
Epoch: 02, Loss: 102.7435
Epoch: 03, Loss: 102.6695
Epoch: 04, Loss: 102.5758
AP SCORE：  0.8700090020746369
AUC SCORE:  0.864
Epoch: 00, Loss: 102.4646
Epoch: 01, Loss: 102.4567
Epoch: 02, Loss: 102.4419
Epoch: 03, Loss: 102.4208
Epoch: 04, Loss: 102.3941
AP SCORE：  0.8700090020746369
AUC SCORE:  0.864
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 102.3624
Epoch: 01, Loss: 102.3352
Epoch: 02, Loss: 102.2835
Epoch: 03, Loss: 102.2099
Epoch: 04, Loss: 102.1168
AP SCORE：  0.8705140525796873
AUC SCORE:  0.8656
Epoch: 00, Loss: 102.0064
Epoch: 01, Loss: 101.9984
Epoch: 02, Loss: 101.9833
Epoch: 03, Loss: 101.9619
Epoch: 04, Loss: 101.9347
AP SCORE：  0.8713473859130207
AUC SCORE:  0.8672
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 101.9025
Epoch: 01, Loss: 101.87

######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 94.4391
Epoch: 01, Loss: 94.4176
Epoch: 02, Loss: 94.3768
Epoch: 03, Loss: 94.3188
Epoch: 04, Loss: 94.2453
AP SCORE：  0.8817641877238334
AUC SCORE:  0.8816
Epoch: 00, Loss: 94.1579
Epoch: 01, Loss: 94.1517
Epoch: 02, Loss: 94.1398
Epoch: 03, Loss: 94.1230
Epoch: 04, Loss: 94.1016
AP SCORE：  0.8817641877238334
AUC SCORE:  0.8816
######################### 模块循环 ： 81 ##########################
Epoch: 00, Loss: 94.0763
Epoch: 01, Loss: 94.0548
Epoch: 02, Loss: 94.0139
Epoch: 03, Loss: 93.9558
Epoch: 04, Loss: 93.8821
AP SCORE：  0.8817641877238334
AUC SCORE:  0.8816
Epoch: 00, Loss: 93.7945
Epoch: 01, Loss: 93.7880
Epoch: 02, Loss: 93.7758
Epoch: 03, Loss: 93.7584
Epoch: 04, Loss: 93.7365
AP SCORE：  0.8828479315661979
AUC SCORE:  0.8832000000000001
######################### 模块循环 ： 82 ##########################
Epoch: 00, Loss: 93.7104
Epoch: 01, Loss: 93.6888
Epoch: 02, Loss: 93.6477
Epoch: 03, Loss: 93.5892
Epo

In [21]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())
"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.826938763251605
STD AP :  0.06899239250454726
MEAN AUC :  0.8384
STD AUC :  0.07150406981424204


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'